In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np



def gausian(img, k):
  # 가우시안 블러 적용
  blurred_img = cv2.GaussianBlur(img, (15, 15), k)
  return blurred_img

def motion(img, kk):

  kernel_size = kk

  # 모션 블러 커널 생성
  kernel_motion_blur = np.zeros((kernel_size, kernel_size))
  kernel_motion_blur[int((kernel_size-1)/2), :] = np.ones(kernel_size)
  kernel_motion_blur = kernel_motion_blur / kernel_size
  output = cv2.filter2D(img, -1, kernel_motion_blur)
  return output


In [ ]:
def white(img, k3):
  h, w = img.shape[:2]

  noise = np.random.normal(0, k3, img.shape).astype(np.float32)

  # 노이즈 추가
  noisy_img = cv2.add(img.astype(np.float32), noise)

  # 이미지를 0~255 범위로 변환
  noisy_img = cv2.normalize(noisy_img, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
  return noisy_img

In [ ]:

def add_light_effect(img, intensity):
    # 랜덤한 색으로 이미지 색상 조정
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(img_hsv)
    v = np.where(v <= 255 - intensity, v + intensity, 255)
    img_hsv = cv2.merge((h, s, v))
    img_color = cv2.cvtColor(img_hsv, cv2.COLOR_HSV2BGR)

    # 모션 블러 효과 적용
    ksize = 15
    kernel = np.zeros((ksize, ksize))
    kernel[int((ksize-1)/2), :] = np.ones(ksize)
    kernel = kernel / ksize
    img_blur = cv2.filter2D(img_color, -1, kernel)

    # 색상 조정된 이미지와 모션 블러 이미지 합성
    img_blend = cv2.addWeighted(img_color, 0.5, img_blur, 0.5, 0)

    return img_blend

In [ ]:
def rot(image, ang):

    # 이미지 중앙점 구하기
    height, width = image.shape[:2]
    center = (width / 2, height / 2)

    # 회전할 각도 설정
    angle = ang

    # 회전 변환 매트릭스 생성
    M = cv2.getRotationMatrix2D(center, angle, 1.0)

    # 이미지 회전
    rotated_image = cv2.warpAffine(image, M, (width, height),borderValue=(255, 255, 255))
    return rotated_image

In [ ]:


#img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
def save(img, path2, name):
  img = cv2.resize(img,(512,512))

  for i in range(0,5):
    rot1=rot(img,-2+i)
    g1 = gausian(img, 1+1*i)
    m1 = motion(img,5+3*i)
    w1 = white(img, 50+50*i)
    r1 = add_light_effect(img,50+30*i)
    cv2.imwrite(path2+'o/'+name+str(i)+'.jpg', rot1)
    cv2.imwrite(path2+'g/'+name+str(i)+'.jpg', g1)
    cv2.imwrite(path2+'m/'+name+str(i)+'.jpg', m1)
    cv2.imwrite(path2+'w/'+name+str(i)+'.jpg', w1)
    cv2.imwrite(path2+'r/'+name+str(i)+'.jpg', r1)

def save2(img, path2, name):
  img = cv2.resize(img,(512,512))

  i=3
  rot1=rot(img,-2+i)
  g1 = gausian(img, 1+1*i)
  m1 = motion(img,5+3*i)
  w1 = white(img, 50+50*i)
  r1 = add_light_effect(img,50+30*i)
  cv2.imwrite(path2+name+'-o_'+str(i)+'.jpg', rot1)
  cv2.imwrite(path2+name+'-g_'+str(i)+'.jpg', g1)
  cv2.imwrite(path2+name+'-m_'+str(i)+'.jpg', m1)
  cv2.imwrite(path2+name+'-w_'+str(i)+'.jpg', w1)
  cv2.imwrite(path2+name+'-r_'+str(i)+'.jpg', r1)



In [ ]:
#저장경로
path2 = '/content/compa/testdata/'

#path3 = '/content/drive/MyDrive/pcbdata/pcb/testdata'
n=0

image_files = get_image_files(folder_path)
for image_file in image_files[40:50]:
    n=n+1
    img = cv2.imread(folder_path + image_file+'.jpg')
    img = cv2.resize(img,(512,512))
    save(img, path2, image_file)
    print(image_file)
    print(n)

In [ ]:
import zipfile
import os

def zip_folder(folder_path, zip_path):
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, _, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, folder_path)
                zipf.write(file_path, arcname)

# 압축 대상 폴더 경로
folder_path = '/content/drive/MyDrive/pcbdata/pcb'

# 압축 파일 경로
zip_path = '/content/drive/MyDrive/pcbdata/pcb3.zip'

# 폴더 압축하여 저장
zip_folder(folder_path, zip_path)